<a href="https://colab.research.google.com/github/luigimersico/Topic_modelling/blob/main/Create_documents_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas pyPDF2 xlsxwriter PyMuPDF keyboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
from PyPDF2 import PdfReader
import csv
import re
import xlsxwriter
import fitz
import pathlib
import keyboard
import time
from pathlib import Path


write a code in python to perform the following activites: 1) read multiple .pdf file of scientific article. Articles have authors, title, year of publications and corpus. 2) create a variable storing the name of the file. 3) extract from the pdf files title, authors, year of publications and corpus 3) dataframe file with 4 columns: 1 ID of article 2 year of publications 3) authors 4) title 5) corpus.  Add the extracted text in the  right column of csv file

In [ ]:
# Specify the directory where your PDF files are located
pdf_folder = '/content/drive/MyDrive/Colab_Notebooks/input_data/reviews_plastics_policies'
# Specify the directory to save the converted text files
text_output_folder = '/content/drive/MyDrive/Colab_Notebooks/output_data'

In [3]:
def get_files(folder):
    files = [f for f in os.listdir(folder) if f.endswith(".pdf")]
    return files

In [7]:
# Get a list of all PDF files in the specified directory
pdf_file_names = get_files(pdf_folder)

In [10]:
def convert_pdf_to_text(pdf_path, output_folder):
    pdf_reader = PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    txt_file_path = os.path.join(output_folder, os.path.splitext(os.path.basename(pdf_path))[0] + ".txt")
    with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [11]:
# Convert each PDF file to text and save in the text output folder
for pdf_file_name in pdf_file_names:
    pdf_file_path = os.path.join(pdf_folder, pdf_file_name)
    convert_pdf_to_text(pdf_file_path, text_output_folder)

In [13]:
# Specify the directory where the converted text files are located
folder_path = '/content/drive/MyDrive/Colab_Notebooks/output_data'

In [14]:
def extract_info_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Extract authors
    authors = os.path.basename(file_path).split('-')[0].strip()
    #authors_pattern = r"Authors:(.*?)(?:\n|$)"
    #authors_match = re.search(authors_pattern, text, re.IGNORECASE | re.DOTALL)
    #authors = authors_match.group(1).strip() if authors_match else ""

    # Extract title
    title = os.path.basename(file_path).split('-')[-1].replace('.txt', '').strip()

    # Extract year
    year_pattern = r"\d{4}"
    year_match = re.search(year_pattern, file_path)
    year = year_match.group(0) if year_match else ""

    # Extract abstract
    abstract_pattern = r"ABSTRACT(.*?)(?:\n\n|1\. Introduction:|$)"
    abstract_match = re.search(abstract_pattern, text, re.IGNORECASE | re.DOTALL)
    abstract = abstract_match.group(1).strip() if abstract_match else ""

    return {
        'Title': title,
        'Authors': authors,
        'Year': year,
        'Abstract': abstract
    }


In [15]:
def main(folder_path):
    extracted_data = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            extracted_info = extract_info_from_txt(file_path)
            extracted_data.append(extracted_info)

    df = pd.DataFrame(extracted_data)
    return df

In [16]:
data_df = main(folder_path)

In [17]:
data_df.head()

,Title,Authors,Year,Abstract
0,Disposal is not free fiscal instruments to intern,Matheson,2022,This paper provides an overview of global soli...
1,A mixed bag The hidden time costs of regulating c,Taylor,2020,The non-monetary costs consumers experienc...
2,Creating a shared vision by uniting local stak...,Neef et al.,2023,
3,Environment tax law to save the planet,Van Den Eijnde,2022,The EU and Member States of the EU have introd...
4,Formulating an Excise Duty on Plastic A Strategy,Irianto et al.,2022,: Plastic excise duty is one of the programs p...
